# Part 1

In [110]:
from gurobipy import *

import numpy as np

In [111]:
node_no = {
    'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7
}

In [112]:
with open('distances.txt', 'r') as file:
    distances = [list(map(float, line.split())) for line in file]

In [113]:
paths = []
with open('paths.txt', 'r') as file:
    for line in file:
        at_node = {}
        path = line.strip().split(': ')[1].split('-')
        actual_path = []
        start = node_no[path[0]]
        end = node_no[path[-1]]
        total_dist = 0
        at_node[0] = path[0]
        for el in range(len(path)-1):
            dest = path[el + 1]
            source = path[el] 
            
            total_dist += distances[node_no[source]][node_no[dest]]
            at_node[total_dist] = path[el+1]
        paths.append({'total_dist':total_dist, 'start':start, 'end': end, 'actual_path':path, 'arrival_times':at_node})

In [114]:
starts_with = {}
for i in range(15):
    for j in range(8):
        starts_with[i, j] = 1 if paths[i]['start'] == j else 0

In [115]:
distances_to_depots = []
with open('depot_node_distances.txt', 'r') as file:
    for line in file:
        distances_to_depots.append([int(x) for x in line.strip().split(': ')[1].split('-')])


In [116]:
depot_no = {
    'X':0, 'Y':1
}

In [117]:
odd_even = {}

for i in range(len(paths)):
        for j in range(2):
            time = 20
            #for X depot
            time -= distances_to_depots[j][paths[i]['start']]
            tour = 0
            while(time > paths[i]['total_dist']):
                time -= paths[i]['total_dist']
                tour += 1
            if tour % 2 == 0: #even number of tours
                time -= distances_to_depots[j][paths[i]['start']]
                if time >= 0:
                    #print("success! begin and end nodes are same for route")
                    odd_even[i, j] = 1
                else:
                    time += paths[i]['total_dist']
                    tour -= 1
                    odd_even[i, j] = 0

                    #print("you need to take one less tour, sorry, ODDIZED")
            else: #odd number of tours
                time -= distances_to_depots[j][paths[i]['end']]
                if time >= 0:
                    #print("success! begin and end nodes are not same for route")
                    odd_even[i, j] = 0
                else:
                    time = time + paths[i]['total_dist']
                    tour -= 1
                    odd_even[i, j] = 1
                    #print("you need to take one less tour, sorry , EVENIZED")
            paths[i][f'tours_taken_{"X" if j == 0 else "Y"}'] = tour
            

In [119]:
X = {}
model = Model('Part1_model')

for i in range(1,16):
    for j in ['X','Y']:
            X[i,j] = model.addVar(vtype = GRB.BINARY,name=f'x_{i}_{j}')

#Constraints
#Trains assigned to deport j starts from node k
for i in range(1,16):
    model.addConstr(quicksum(X[i,j] for j in ['X','Y']) == 1)
     
# X -> A - max 3        
#A route cannot be used more than 3 trains
for j in ['X','Y']:
    for k in range(0,8):
        model.addConstr(quicksum(starts_with[i-1,k]*X[i,j] for i in range(1,16)) <= 3)
        
#Each depot shpuld have at least 5 trains assigned to it
for j in ['X','Y']:
    model.addConstr(quicksum(X[i,j] for i in range(1,16)) >= 5)
        
#Every train should have only one starting node
#for k in range(1,9):
#    model.addConstr(quicksum(X[i,j,k] for i in range(1,16) for j in ['X','Y']) == 1)


objective_func = quicksum((1 + odd_even[i - 1, depot_no[j]]) * X[i,j] * distances_to_depots[depot_no[j]][paths[i-1]['start']] + 
                          (1 - odd_even[i - 1, depot_no[j]]) * X[i,j] * distances_to_depots[depot_no[j]][paths[i-1]['end']] 
                          for i in range(1,16) for j in ['X', 'Y'])
model.setObjective(objective_func, GRB.MINIMIZE)

model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.4.0 22E252)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 33 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x5c5126fd
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 48.0000000
Presolve removed 27 rows and 18 columns
Presolve time: 0.00s
Presolved: 6 rows, 12 columns, 38 nonzeros
Variable types: 0 continuous, 12 integer (9 binary)
Found heuristic solution: objective 34.0000000

Root relaxation: objective 3.300000e+01, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

In [120]:
assigned = {}
for j in ['X', 'Y']:
    assigned_trains = [i for i in range(1, 16) if X[i, j].X > 0.5]
    assigned[j] = assigned_trains
    print(f"Depot {j} assigned trains: {assigned_trains}")

Depot X assigned trains: [1, 2, 6, 8, 9, 11, 12, 15]
Depot Y assigned trains: [3, 4, 5, 7, 10, 13, 14]


# Part 2

In [121]:
#read parameters.txt
other_params = {}

with open('parameters.txt', 'r') as file:
    for line in file:
        other_params[line.strip().split(': ')[0]] = line.strip().split(': ')[1].split("$")[0]


In [122]:
for i in assigned['X']:    
    paths[i-1].pop('tours_taken_Y', None)
for i in assigned['Y']:    
    paths[i-1].pop('tours_taken_X', None)
    

In [123]:
for i in range(len(paths)):
    arr_times = paths[i]['arrival_times']
    start_depot = 0 if 'tours_taken_X' in paths[i] else 1
    first_dis = distances_to_depots[start_depot][paths[i]['start']]
    
    tours = paths[i]['tours_taken_X'] if 'tours_taken_X' in paths[i] else paths[i]['tours_taken_Y']
    
    actual_path = paths[i]['actual_path']
    
    new_path = actual_path
    for j in range(tours-1):
        if j % 2 == 0:
            new_path = new_path + actual_path[::-1][1:]
        else:
            new_path = new_path + actual_path[1:]
    
    arr_times_new = {}
    arr_times_new[0] = 'X' if start_depot == 0 else 'Y'
    
    first_dis = distances_to_depots[start_depot][paths[i]['start']]
    
    total_dist = 0
    arr_times_new[first_dis] = new_path[0]
    
    for el in range(len(new_path)-1):
            dest = new_path[el + 1]
            source = new_path[el] 
            
            total_dist += distances[node_no[source]][node_no[dest]]
            arr_times_new[total_dist + first_dis] = new_path[el+1]
    if(odd_even[i, start_depot] == 1):
        arr_times_new[total_dist + 2 * first_dis] = 'X' if start_depot == 0 else 'Y'
    else:
        arr_times_new[total_dist + first_dis + distances_to_depots[start_depot][paths[i]['end']]] = 'X' if start_depot == 0 else 'Y'
    
    total_dur = list(arr_times_new)[-1]
    for k in range(25):
        if k not in arr_times_new:
            arr_times_new[k] = 'OW' # On way
        if k > total_dur or k > 20:
            arr_times_new[k] = 'MA' # Maintenance 
    
    # 21: 'MA' means maintenance from 20 to 21        
    paths[i]['train_locs'] = dict(sorted(arr_times_new.items()))
    paths[i]['path_after_tours'] = new_path

paths

[{'total_dist': 7.0,
  'start': 0,
  'end': 1,
  'actual_path': ['A', 'C', 'H', 'B'],
  'arrival_times': {0: 'A', 2.0: 'C', 4.0: 'H', 7.0: 'B'},
  'tours_taken_X': 2,
  'train_locs': {0: 'X',
   1: 'A',
   2: 'OW',
   3.0: 'C',
   4: 'OW',
   5.0: 'H',
   6: 'OW',
   7: 'OW',
   8.0: 'B',
   9: 'OW',
   10: 'OW',
   11.0: 'H',
   12: 'OW',
   13.0: 'C',
   14: 'OW',
   15.0: 'A',
   16.0: 'X',
   17: 'MA',
   18: 'MA',
   19: 'MA',
   20: 'MA',
   21: 'MA',
   22: 'MA',
   23: 'MA',
   24: 'MA'},
  'path_after_tours': ['A', 'C', 'H', 'B', 'H', 'C', 'A']},
 {'total_dist': 3.0,
  'start': 1,
  'end': 0,
  'actual_path': ['B', 'G', 'A'],
  'arrival_times': {0: 'B', 1.0: 'G', 3.0: 'A'},
  'tours_taken_X': 6,
  'train_locs': {0: 'X',
   1: 'B',
   2.0: 'G',
   3: 'OW',
   4.0: 'A',
   5: 'OW',
   6.0: 'G',
   7.0: 'B',
   8.0: 'G',
   9: 'OW',
   10.0: 'A',
   11: 'OW',
   12.0: 'G',
   13.0: 'B',
   14.0: 'G',
   15: 'OW',
   16.0: 'A',
   17: 'OW',
   18.0: 'G',
   19.0: 'B',
   20.0: 'X'

In [66]:
Y = {}
model2 = Model('Part2_model')
# number of type i trains purchased

# number of in depot charging station built at depot i
# number of on route charging station built at station i